# Create a Custom HSV Profile

This step is needed in order to ensure the best possible thresholding for skin tones in the image and should be manually configured once before using.  
The default profile is derived from statistical data.  

Be sure to be in the root directory of the project before running the following code boxes.

We first import all the needed dependencies to run the script.

In [6]:
import cv2 
import numpy as np
from IPython.display import clear_output
import tkinter as tk
import ast
import os
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib.widgets import RangeSlider
from PyQt5 import QtGui

We then define two callback functions, linked to pressing the close button of the window and on keyboard interrupt.
Pressing the close button or pressing "Q" on the keyboard will simply close the program without saving the config.dat file.
Pressing "E" on the keyboard will instead save the config.dat file with the current HSV profile before closing.

In [7]:
# Window close handler
def handle_close(event, cap):
    cap.release()
    plt.close('all')

# Keyboard interrupt handler
def on_press(event):
    if event.key == 'q':
        print("You pressed " + event.key + ", the program exited without saving")
        cap.release()
        plt.close('all')
    elif event.key == 'e':
        print("You pressed " + event.key + ", the program saved and exited")
        cap.release()
        file = open("Config\config.dat", "w")
        configToStr = repr(config)
        file.write(configToStr + "\n")
        file.close()
        plt.close('all')

In [8]:
# Setup camera capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
root = tk.Tk()
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()

The following code box will generate the MatPlotLib window without displaying any output. You can check that the config.dat file has been correctly read before moving on. If no cofig.dat file is found, the program will provide a default statistically-relevant HSV skin thresholding model.

In [9]:
# Check if a configuration file exists, else load a predefined value set
if os.path.isfile('Config\config.dat'):
    print("Configuration file found\n")
    file = open("Config\config.dat", "r")
    contents = file.read()
    config = ast.literal_eval(contents)
    file.close()
else:
    print ("Configuration file not found\n")
    config = {'HL': 0, 'SL': 29, 'VL': 24, 'HH': 40, 'SH': 255, 'VH': 255}        

# Setup window
plt.ion()
imageWin = None
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
plt.subplots_adjust(bottom=0.25)
fig.canvas.mpl_connect("close_event", lambda event: handle_close(event, cap))
fig.canvas.mpl_connect('key_press_event', on_press)
plt.get_current_fig_manager().set_window_title('HSV Custom Profile Generator')
title_obj = plt.title('HSV Custom Profile Generator')
plt.setp(title_obj, color='#d4d4d4')
fig.patch.set_facecolor('#1e1e1e')
fig.patch.set_edgecolor('#1e1e1e')

# Create the RangeSliders
sliderAxHue = plt.axes([0.20, 0.2, 0.60, 0.03])
sliderHue = RangeSlider(sliderAxHue, "Hue", valmin=0, valmax=255, valstep = 1)
sliderHue.set_val((config["HL"], config["HH"]))
sliderAxSat = plt.axes([0.20, 0.15, 0.60, 0.03])
sliderSat = RangeSlider(sliderAxSat, "Sat", valmin=0, valmax=255, valstep = 1)
sliderSat.set_val((config["SL"], config["SH"]))
sliderAxVal = plt.axes([0.20, 0.1, 0.60, 0.03])
sliderVal = RangeSlider(sliderAxVal, "Val", valmin=0, valmax=255, valstep = 1)
sliderVal.set_val((config["VL"], config["VH"]))

Configuration file found



The following code box contains the main camera loop and displays the original camera frame on the left, the HSV frame thresholded according to the values read in the config dictionary on the right, other than 3 range sliders on the bottom, to be used in order to select the desired values for the Hue, Saturation and Value paramenters.

In [10]:
# Calibration loop
while cap.isOpened(): 
    ret, frame = cap.read()
    
    # Convert the frame to HSV and read trackbar inputs
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

    # Read RangeSliders' values
    config["HL"] = int(sliderHue.val[0])
    config["HH"] = int(sliderHue.val[1])
    config["SL"] = int(sliderSat.val[0])
    config["SH"] = int(sliderSat.val[1])
    config["VL"] = int(sliderVal.val[0])
    config["VH"] = int(sliderVal.val[1])
    
    # Perform thresholding according to the specified values
    thresh = cv2.inRange(hsv, (config["HL"], config["SL"], config["VL"]), (config["HH"], config["SH"], config["VH"]))
    
    # Force redraw
    fig.canvas.draw_idle()

    # Show images
    if imageWin is None:
        plt.subplot(1, 2, 1).axis("off")
        imageWin = plt.imshow(frame, "gray")
        title_obj = plt.title("Original Frame")
        plt.setp(title_obj, color='#d4d4d4')
        plt.subplot(1, 2, 2).axis("off")
        threshWin = plt.imshow(thresh, "gray")
        title_obj = plt.title("Thresholded Frame")
        plt.setp(title_obj, color='#d4d4d4')
        plt.show()
    else:
        imageWin.set_data(frame)
        threshWin.set_data(thresh)
        fig.canvas.draw()
        fig.canvas.flush_events() 

You pressed e, the program saved and exited
